### Obesity - Classification (Semester B)


In [91]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import magma
import matplotlib.cm as cm 
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')


np.set_printoptions(precision=5)
#show 5 number after decimal point 

In [92]:
df = pd.read_csv("ObesityDataSet_raw_and_data_sinthetic.csv")
df.head(3)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight


בסמסטר הקודם הורדנו רק את עמודת המשקל מכיוון שייש לה קשר ישיר ל-בי-אמ-איי. בשיפור העבודה ננסה להוריד גם את עומדת הגובה שכן גם לה יש קשר חזק ל-בי-אמ-איי.
בסמסטר הקודם המודלים שלנו בעיקר התקשו להגדיר מהו אדם רגיל ומהו אדם עם אוביסטי סוג 3. 
ננסה להבין מה יוצר את הקושי וכימד נוכל לעזור למודלים שלנו לסווג בצורה טובה יותר.
בסמסטר הקודם, מדד הדיוק הגבוהה ביותר שהיה לנו הוא 81%. ננסה להשיג ציון טוב יותר או לפחות לנסות שמודלים שלנו לא יטעו בצורה קיצונית (כמו להגדיר אדם כאובוסטי סוג 3 כאשר בפועל הוא בתת משקל)


In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [94]:
#Other than 'Age', 'Weight' and 'Height' there is no reason for the other numeric columns to be 'float'. We'll change it.
columns = ["NCP", "CH2O", "TUE", "FAF","FCVC"]
for col in columns:
    df[col] = round(df[col]).astype('int')

In [95]:
df_res = df.copy(deep=True)
df_res = pd.get_dummies(df, columns=['Gender','family_history_with_overweight','FAVC','SMOKE',
                                     'SCC','MTRANS','FCVC','NCP','CAEC','CH2O','FAF','TUE','CALC'])
df_res.head(3)

,Age,Height,Weight,NObeyesdad,Gender_Female,Gender_Male,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,...,FAF_1,FAF_2,FAF_3,TUE_0,TUE_1,TUE_2,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no
0,21.0,1.62,64.0,Normal_Weight,1,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,1
1,21.0,1.52,56.0,Normal_Weight,1,0,0,1,1,0,...,0,0,1,1,0,0,0,0,1,0
2,23.0,1.80,77.0,Normal_Weight,0,1,0,1,1,0,...,0,1,0,0,1,0,0,1,0,0


In [96]:
df_res.drop(columns=['Weight'],inplace=True)

In [97]:
from sklearn.model_selection import train_test_split
X = df_res.drop(columns=["NObeyesdad"])
y = df_res['NObeyesdad'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [98]:
from sklearn.preprocessing import MinMaxScaler
norm = MinMaxScaler()
X_train_norm = norm.fit_transform(X_train)
X_test_norm = norm.transform(X_test)

In [99]:
# Before we try real models, we will try a dummyClassifier. we will try to get high score than the dummyModel.
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=0)
dummy_clf.fit(X_train_norm, y_train)
dummy_clf.predict(X_train_norm)
print("The DummyClassifier score is:" ,dummy_clf.score(X_test_norm, y_test))

The DummyClassifier score is: 0.1608832807570978


In [100]:
from sklearn.metrics import classification_report
from sklearn import metrics

In [102]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier().fit(X_train_norm, y_train)
y_pred = KNN.predict(X_test_norm)
norm_preci = round(metrics.precision_score(y_test, y_pred, average="weighted"),3)            
norm_accuracy = round(metrics.accuracy_score(y_test, y_pred),3)
print("precision_score:", norm_preci)
print("Accuracy:", norm_accuracy)
print("Classification Report: \n", metrics.classification_report(y_test, y_pred))

precision_score: 0.717
Accuracy: 0.711
Classification Report: 
                      precision    recall  f1-score   support

Insufficient_Weight       0.72      0.88      0.80        86
      Normal_Weight       0.78      0.33      0.47        93
     Obesity_Type_I       0.65      0.76      0.70       102
    Obesity_Type_II       0.64      0.90      0.75        88
   Obesity_Type_III       0.84      0.99      0.91        98
 Overweight_Level_I       0.71      0.53      0.61        88
Overweight_Level_II       0.66      0.54      0.60        79

           accuracy                           0.71       634
          macro avg       0.72      0.71      0.69       634
       weighted avg       0.72      0.71      0.69       634



In [103]:
from xgboost import XGBClassifier

XGB = XGBClassifier(objective="multi:softmax",num_class= 7, random_state=42)
XGB.fit(X_train_norm, y_train, eval_metric='auc')
y_pred_XGB = XGB.predict(X_test_norm)

accuracy = metrics.accuracy_score(y_test, y_pred_XGB)
print("Accuracy:", accuracy)
print("Classification Report: \n", metrics.classification_report(y_test, y_pred_XGB))

Accuracy: 0.8091482649842271
Classification Report: 
                      precision    recall  f1-score   support

Insufficient_Weight       0.79      0.88      0.84        86
      Normal_Weight       0.63      0.61      0.62        93
     Obesity_Type_I       0.81      0.81      0.81       102
    Obesity_Type_II       0.84      0.94      0.89        88
   Obesity_Type_III       0.97      0.99      0.98        98
 Overweight_Level_I       0.78      0.72      0.75        88
Overweight_Level_II       0.82      0.68      0.74        79

           accuracy                           0.81       634
          macro avg       0.81      0.81      0.80       634
       weighted avg       0.81      0.81      0.81       634



In [104]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 

DTC = DecisionTreeClassifier()
KNN = KNeighborsClassifier()
log_clf = LogisticRegression()

voting_clf = VotingClassifier(
 estimators=[('lr', log_clf),('knn',KNN),('dt', DTC)],
 voting='hard')

voting_clf.fit(X_train, y_train)


VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('knn', KNeighborsClassifier()),
                             ('dt', DecisionTreeClassifier())])

In [105]:
from sklearn.metrics import accuracy_score
for clf in (log_clf,  KNN,DTC, voting_clf):
    clf.fit(X_train_norm, y_train)
    y_pred = clf.predict(X_test_norm)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))


LogisticRegression 0.5851735015772871
KNeighborsClassifier 0.7113564668769716
DecisionTreeClassifier 0.7145110410094637
VotingClassifier 0.7192429022082019


In [107]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier().fit(X_train_norm, y_train)
y_pred = GBC.predict(X_test_norm)

accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report: \n", metrics.classification_report(y_test, y_pred)) 

Accuracy: 0.7570977917981072
Classification Report: 
                      precision    recall  f1-score   support

Insufficient_Weight       0.79      0.83      0.81        86
      Normal_Weight       0.59      0.59      0.59        93
     Obesity_Type_I       0.70      0.75      0.73       102
    Obesity_Type_II       0.84      0.92      0.88        88
   Obesity_Type_III       0.94      0.99      0.97        98
 Overweight_Level_I       0.68      0.60      0.64        88
Overweight_Level_II       0.73      0.58      0.65        79

           accuracy                           0.76       634
          macro avg       0.75      0.75      0.75       634
       weighted avg       0.75      0.76      0.75       634



In [108]:
#Searching for better performance out of the models with RandomizedSearch
from sklearn.model_selection import RandomizedSearchCV

In [115]:
#XGBClassifier

parameters = {
        'learning_rate': [ 0.1,0.3,0.03],
        'max_depth': [6,10,15,20],
        'min_child_weight': [1, 2],
        'subsample': [0.5,0.6,1],
        'colsample_bytree': [0.3,0.5, 0.7],
        'n_estimators' : [100,150,170,200]
         }

random_XGB = RandomizedSearchCV(XGBClassifier(eval_metric='acu'),parameters, n_jobs=-1, 
                          random_state=42, n_iter = 50, verbose = True)
random_XGB.fit(X_train_norm,y_train, eval_metric='auc')
y_pred_XGB_random = random_XGB.predict(X_test_norm)
print ("Accuracy Score = ",metrics.accuracy_score(y_test,y_pred_XGB_random))
print (random_XGB.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Accuracy Score =  0.8138801261829653
{'subsample': 1, 'n_estimators': 150, 'min_child_weight': 1, 'max_depth': 20, 'learning_rate': 0.3, 'colsample_bytree': 0.7}
